### Cost model benchmarking - case study #1

Loosely based on Dudgeon Wind Farm (400 MW, 8MW turbines)

National Renewable Energy Lab

Matt Shields, Jake Nunemaker

Updated: 2/17/2020

In [1]:
import os
import sys
import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as plt_dates

import ORBIT
print(f"Using ORBIT version {ORBIT.__version__}.")
from ORBIT import ProjectManager


usd_to_euro = 0.92

Using ORBIT version 0+unknown.


#### Weather File

In [2]:
filepath = os.path.join('library', 'weather', 'ERA5_1979_2019.csv')
weather = pd.read_csv(filepath, parse_dates=["datetime"]).set_index(keys='datetime')
weather.describe()

,windspeed,waveheight
count,354984.000000,354984.000000
mean,9.346687,1.186658
std,4.432092,0.708619
min,0.032738,0.055604
25%,6.047695,0.676800
50%,9.080866,1.025000
75%,12.285874,1.517500
max,31.712409,6.804200


#### ORBIT Configuration

In [3]:
phases = [
    # Substructures
    'MonopileInstallation',
#     'ScourProtectionDesign',
#     'ScourProtectionInstallation',
    
    # Turbines
    'TurbineInstallation',
    
#     # Electrical
    'ArraySystemDesign',
    'ArrayCableInstallation',
    'ExportSystemDesign',
    'ExportCableInstallation',
    'OffshoreSubstationDesign',
    'OffshoreSubstationInstallation'
]

required_config = ProjectManager.compile_input_dict(phases)
# required_config

In [4]:
config = {
    # Substations
#     "num_substations": 1,
    
    # Vessels
#     'scour_protection_install_vessel': 'ExampleScour',
#     'trench_dig_vessel': 'StematSpirit',
#     'array_cable_lay_vessel': 'cable_lay_vessel',
#     'export_cable_lay_vessel': 'StematSpirit',
#     "oss_install_vessel": "OlegStrashnov",  # Actually vessel from SPT Offshore
    
    # Site/plant
    'site': {
        'depth': 22.5,
        'distance': 124,
        'distance_to_landfall': 42,
        'distance_to_beach': 0,
        'distance_to_interconnection': 3,
        'mean_windspeed': 9.13
    },
    
    'plant': {
        'layout': 'grid',
        'num_turbines': 50,
        'row_spacing': 7,
        'turbine_spacing': 9,
        'substation_distance': 1
    },
    
    'port': {
        'num_cranes': 1,
        'monthly_rate': 0,#2000000,
        "name": "Green Port"
    },
    
    # Turbine + components
    'turbine': '8MW_generic',

    # Substructure components
    'substructure': {'diameter': 7.2},
    'monopile': {
        'type': 'Monopile',
        'length': 69,
        'diameter': 7.2,
        'deck_space': 496.8,
        'mass': 800,
        'monopile_steel_cost' : 2000 / usd_to_euro,
    },
    
    'transition_piece': {
        'type': 'Transition Piece',
        'deck_space': 100,
        'mass': 400,
        'transition_piece_steel_cost' : 2000 / usd_to_euro,
    },
    
    'scour_protection_design': {
        'cost_per_tonne': 40,
#         'scour_protection_depth':0.3
    },
    
    # Electrical
    'array_system_design': {
        'cables': 'XLPE_630mm_66kV'
    },
    
    'array_system': {
        'strategy': 'lay_bury'
    },
    
    'export_system': {
        'strategy': 'lay_bury'
    },
    
    'export_system_design': {
        'cables': 'XLPE_1000mm_220kV',
        'percent_added_length': 0
    },
    
    'substation_design': {
        'num_substations': 2
    },
    
#     "offshore_substation_topside": {
#         "type": "Topside",
#         "deck_space": 200,
#         "weight": 2000,
#     },
#     "offshore_substation_substructure": {
#         "type": "Monopile",
#         "deck_space": 500,
#         "weight": 1850,
#         "length": 69,  # Assumed to be the same as monopile length
#     },


    # Phase specific configurations
    'MonopileInstallation': {
        'wtiv': 'Benchmarking_WTIV_turbine',  
        
    },
    
    'TurbineInstallation': {
        'wtiv': 'Benchmarking_WTIV_turbine'
    },
    
    'ArrayCableInstallation': {
        'array_cable_install_vessel' : 'cable_lay_vessel'
    }, 
    
    'ExportCableInstallation': {
        'export_cable_install_vessel' : 'cable_lay_vessel'
    },
    
    'OffshoreSubstationInstallation': {
        "num_feeders": 2,
        "feeder": "zero_cost_large_feeder",
        "oss_install_vessel": "Benchmarking_WTIV_turbine"
    },    
    
    'ScourProtectionInstallation': {
        'spi_vessel': 'example_scour_protection_vessel'
    },
    
    # Phases
    'design_phases': [
#         "MonopileDesign",
        'ScourProtectionDesign',
        'ArraySystemDesign',
        'ExportSystemDesign',
        'OffshoreSubstationDesign'
    ],
    
    'install_phases': {
        'MonopileInstallation': '07/01/2000',         # Updated dates 
        'ScourProtectionInstallation': '07/01/2000',  # Placed at the end of the monopile installation
        'TurbineInstallation': '07/01/2000',
        'ArrayCableInstallation': '07/01/2000',
        'ExportCableInstallation': '07/01/2000',
        'OffshoreSubstationInstallation': '07/01/2000'
    }
}

In [5]:
path = os.path.join(os.getcwd(), "library")
project = ProjectManager(config, weather=weather, library_path=path)

ORBIT library intialized at 'C:\Users\mshields\Documents\Projects\ORBIT analysis\Verification - Benchmarking\library'


In [6]:
project.run_project()

# Results

### Turbine installation

In [7]:
# Trip data
project._phases['TurbineInstallation'].wtiv.trip_data

[Trip(cargo_mass=3012, deck_space=3884, items={TowerSection: 8, Nacelle: 4, Blade: 12}),
 Trip(cargo_mass=3012, deck_space=3884, items={TowerSection: 8, Nacelle: 4, Blade: 12}),
 Trip(cargo_mass=3012, deck_space=3884, items={TowerSection: 8, Nacelle: 4, Blade: 12}),
 Trip(cargo_mass=3012, deck_space=3884, items={TowerSection: 8, Nacelle: 4, Blade: 12}),
 Trip(cargo_mass=3012, deck_space=3884, items={TowerSection: 8, Nacelle: 4, Blade: 12}),
 Trip(cargo_mass=3012, deck_space=3884, items={TowerSection: 8, Nacelle: 4, Blade: 12}),
 Trip(cargo_mass=3012, deck_space=3884, items={TowerSection: 8, Nacelle: 4, Blade: 12}),
 Trip(cargo_mass=3012, deck_space=3884, items={TowerSection: 8, Nacelle: 4, Blade: 12}),
 Trip(cargo_mass=3012, deck_space=3884, items={TowerSection: 8, Nacelle: 4, Blade: 12}),
 Trip(cargo_mass=3012, deck_space=3884, items={TowerSection: 8, Nacelle: 4, Blade: 12}),
 Trip(cargo_mass=3012, deck_space=3884, items={TowerSection: 8, Nacelle: 4, Blade: 12}),
 Trip(cargo_mass=3012

In [8]:
# Installation costs 
turb_time = np.round(project._phases['TurbineInstallation'].total_phase_time * (1/24) ,1 ) # Convert hours to days
turb_cost = np.round(project._phases['TurbineInstallation'].total_phase_cost * 1e-6 * usd_to_euro, 1)
turb_delay = np.round(turb_time * (1-project._phases['TurbineInstallation'].detailed_output['TurbineInstallation']['WTIV_operational_efficiency']), 1)

print('Turbine install time: ', turb_time, 'days')
print('Turbine install cost: ', turb_cost, 'M Euros')
print('Weather delays during turbine install', turb_delay, 'days')

Turbine install time:  151.2 days
Turbine install cost:  31.2 M Euros
Weather delays during turbine install 9.3 days


In [9]:
proj_df = pd.DataFrame(project.project_actions)
proj_df.loc[proj_df['phase']=='TurbineInstallation']

,action,agent,cable_lay_speed,cost,cost_multiplier,duration,hub_height,level,location,max_waveheight,max_windspeed,phase,phase_name,site_depth,time,transit_speed
2,Mobilize,WTIV,NaN,1.085000e+06,1.0,120.000000,NaN,ACTION,NaN,NaN,NaN,TurbineInstallation,NaN,NaN,0.000000,NaN
10,Fasten Tower Section,WTIV,NaN,3.616667e+04,NaN,4.000000,113.0,ACTION,NaN,NaN,NaN,TurbineInstallation,TurbineInstallation,22.5,4.000000,NaN
16,Fasten Tower Section,WTIV,NaN,3.616667e+04,NaN,4.000000,113.0,ACTION,NaN,NaN,NaN,TurbineInstallation,TurbineInstallation,22.5,8.000000,NaN
19,Fasten Nacelle,WTIV,NaN,3.616667e+04,NaN,4.000000,113.0,ACTION,NaN,NaN,NaN,TurbineInstallation,TurbineInstallation,22.5,12.000000,NaN
23,Fasten Blade,WTIV,NaN,1.356250e+04,NaN,1.500000,113.0,ACTION,NaN,NaN,NaN,TurbineInstallation,TurbineInstallation,22.5,13.500000,NaN
27,Fasten Blade,WTIV,NaN,1.356250e+04,NaN,1.500000,113.0,ACTION,NaN,NaN,NaN,TurbineInstallation,TurbineInstallation,22.5,15.000000,NaN
29,Fasten Blade,WTIV,NaN,1.356250e+04,NaN,1.500000,113.0,ACTION,NaN,NaN,NaN,TurbineInstallation,TurbineInstallation,22.5,16.500000,NaN
33,Fasten Tower Section,WTIV,NaN,3.616667e+04,NaN,4.000000,113.0,ACTION,NaN,NaN,NaN,TurbineInstallation,TurbineInstallation,22.5,20.500000,NaN
35,Fasten Tower Section,WTIV,NaN,3.616667e+04,NaN,4.000000,113.0,ACTION,NaN,NaN,NaN,TurbineInstallation,TurbineInstallation,22.5,24.500000,NaN
41,Fasten Nacelle,WTIV,NaN,3.616667e+04,NaN,4.000000,113.0,ACTION,NaN,NaN,NaN,TurbineInstallation,TurbineInstallation,22.5,28.500000,NaN


### Monopile design and installation

In [10]:
# Trip data
project._phases['MonopileInstallation'].wtiv.trip_data

[Trip(cargo_mass=7200, deck_space=3580.8, items={Monopile: 6, TransitionPiece: 6}),
 Trip(cargo_mass=7200, deck_space=3580.8, items={Monopile: 6, TransitionPiece: 6}),
 Trip(cargo_mass=7200, deck_space=3580.8, items={Monopile: 6, TransitionPiece: 6}),
 Trip(cargo_mass=7200, deck_space=3580.8, items={Monopile: 6, TransitionPiece: 6}),
 Trip(cargo_mass=7200, deck_space=3580.8, items={Monopile: 6, TransitionPiece: 6}),
 Trip(cargo_mass=7200, deck_space=3580.8, items={Monopile: 6, TransitionPiece: 6}),
 Trip(cargo_mass=7200, deck_space=3580.8, items={Monopile: 6, TransitionPiece: 6}),
 Trip(cargo_mass=7200, deck_space=3580.8, items={Monopile: 6, TransitionPiece: 6}),
 Trip(cargo_mass=2400, deck_space=1193.6, items={Monopile: 2, TransitionPiece: 2})]

In [11]:
# Installation and capital costs 
mono_time = np.round(project._phases['MonopileInstallation'].total_phase_time * (1/24) , 1) # Convert hours to days
mono_install_cost = np.round(project._phases['MonopileInstallation'].total_phase_cost * 1e-6 * usd_to_euro, 1)
mono_capital_cost = np.round(((config['monopile']['mass'] * 
                     config['plant']['num_turbines'] * 
                     config['monopile']['monopile_steel_cost']) +
                     (config['transition_piece']['mass'] * 
                     config['plant']['num_turbines'] * 
                     config['transition_piece']['transition_piece_steel_cost']))* 1e-6 * usd_to_euro , 1)
mono_delay = np.round(mono_time * (1-project._phases['MonopileInstallation'].detailed_output['MonopileInstallation']['WTIV_operational_efficiency']), 1)

print('Monopile install time: ', mono_time, 'days')
print('Monopile install cost: ', mono_install_cost, 'M Euros')
print('Monopile capital cost: ', mono_capital_cost, 'M Euros')
print('Monopile total cost: ', mono_capital_cost + mono_install_cost, 'M Euros')
print('Weather delays during monopile install', mono_delay, 'days')

Monopile install time:  86.9 days
Monopile install cost:  18.3 M Euros
Monopile capital cost:  120.0 M Euros
Monopile total cost:  138.3 M Euros
Weather delays during monopile install 1.3 days


In [12]:
proj_df = pd.DataFrame(project.project_actions)
proj_df.loc[proj_df['phase']=='MonopileInstallation']

,action,agent,cable_lay_speed,cost,cost_multiplier,duration,hub_height,level,location,max_waveheight,max_windspeed,phase,phase_name,site_depth,time,transit_speed
0,Mobilize,WTIV,NaN,1.085000e+06,1.0,120.000000,NaN,ACTION,NaN,NaN,NaN,MonopileInstallation,NaN,NaN,0.000000,NaN
18,Fasten Monopile,WTIV,NaN,1.085000e+05,NaN,12.000000,113.0,ACTION,NaN,NaN,NaN,MonopileInstallation,MonopileInstallation,22.5,12.000000,NaN
32,Fasten Transition Piece,WTIV,NaN,7.233333e+04,NaN,8.000000,113.0,ACTION,NaN,NaN,NaN,MonopileInstallation,MonopileInstallation,22.5,20.000000,NaN
46,Fasten Monopile,WTIV,NaN,1.085000e+05,NaN,12.000000,113.0,ACTION,NaN,NaN,NaN,MonopileInstallation,MonopileInstallation,22.5,32.000000,NaN
59,Fasten Transition Piece,WTIV,NaN,7.233333e+04,NaN,8.000000,113.0,ACTION,NaN,NaN,NaN,MonopileInstallation,MonopileInstallation,22.5,40.000000,NaN
77,Fasten Monopile,WTIV,NaN,1.085000e+05,NaN,12.000000,113.0,ACTION,NaN,NaN,NaN,MonopileInstallation,MonopileInstallation,22.5,52.000000,NaN
86,Fasten Transition Piece,WTIV,NaN,7.233333e+04,NaN,8.000000,113.0,ACTION,NaN,NaN,NaN,MonopileInstallation,MonopileInstallation,22.5,60.000000,NaN
106,Fasten Monopile,WTIV,NaN,1.085000e+05,NaN,12.000000,113.0,ACTION,NaN,NaN,NaN,MonopileInstallation,MonopileInstallation,22.5,72.000000,NaN
114,Fasten Transition Piece,WTIV,NaN,7.233333e+04,NaN,8.000000,113.0,ACTION,NaN,NaN,NaN,MonopileInstallation,MonopileInstallation,22.5,80.000000,NaN
128,Fasten Monopile,WTIV,NaN,1.085000e+05,NaN,12.000000,113.0,ACTION,NaN,NaN,NaN,MonopileInstallation,MonopileInstallation,22.5,92.000000,NaN


### Array cable design and installation

In [13]:
# Trip data
# project._phases['ArrayCableInstallation'].install_vessel.__dict__
# project._phases.keys()
# project._phases['ArraySystemDesign'].__dict__

# project._phases['ArraySystemDesign'].detailed_output
project._phases['ArrayCableInstallation'].detailed_output

{'ArrayCableInstallation': {'Array_Cable_Installation_Vessel_operational_efficiency': 0.9497412005935562}}

In [14]:
# Installation and capital costs 
ac_time = np.round(project._phases['ArrayCableInstallation'].total_phase_time * (1/24) , 1) # Convert hours to days
ac_install_cost = np.round(project._phases['ArrayCableInstallation'].total_phase_cost * 1e-6 * usd_to_euro, 1)
ac_capital_cost = np.round(project._phases['ArraySystemDesign'].total_phase_cost * 1e-6 * usd_to_euro, 1)
total_length = np.round(project._phases['ArraySystemDesign'].detailed_output['array_system_length_by_type']['XLPE_630mm_66kV'], 1)
ac_delay = np.round(ac_time * (1-project._phases['ArrayCableInstallation'].detailed_output['ArrayCableInstallation']['Array_Cable_Installation_Vessel_operational_efficiency']), 1)


print('Array cable total length: ',total_length , 'km')
print('Array cable install time: ', ac_time, 'days')
print('Array cable install cost: ', ac_install_cost, 'M Euros')
print('Array cable capital cost: ', ac_capital_cost, 'M Euros')
print('Array cable total cost: ', ac_capital_cost + ac_install_cost, 'M Euros')
print('Weather delays during array cable install', ac_delay, 'days')

Array cable total length:  84.5 km
Array cable install time:  72.9 days
Array cable install cost:  9.7 M Euros
Array cable capital cost:  31.6 M Euros
Array cable total cost:  41.3 M Euros
Weather delays during array cable install 3.7 days


### Export cable 

In [15]:
project._phases['ExportSystemDesign'].detailed_output

{'export_system': {'interconnection_distance': 3,
  'cable': {'linear_density': 80, 'sections': [45.0225], 'number': 2}},
 'export_system_total_mass': 7203.6,
 'export_system_total_length': 90.045,
 'export_system_total_cost': 53756865.0,
 'export_system_cable_power': 342.9460597040872}

In [16]:
# Installation and capital costs 
ec_time = np.round(project._phases['ExportCableInstallation'].total_phase_time * (1/24) , 1) # Convert hours to days
ec_install_cost = np.round(project._phases['ExportCableInstallation'].total_phase_cost * 1e-6 * usd_to_euro, 1)
ec_capital_cost = np.round(project._phases['ExportSystemDesign'].total_phase_cost * 1e-6 * usd_to_euro, 1)
ec_total_length = np.round(project._phases['ExportSystemDesign'].detailed_output['export_system_total_length'], 1)
ec_delay = np.round(ec_time * (1-project._phases['ExportCableInstallation'].detailed_output['ExportCableInstallation']['Export_Cable_Installation_Vessel_operational_efficiency']), 1)


print('Export cable total length: ',ec_total_length , 'km')
print('Export cable install time: ', ec_time, 'days')
print('Export cable install cost: ', ec_install_cost, 'M Euros')
print('Export cable capital cost: ', ec_capital_cost, 'M Euros')
print('Export cable total cost: ', ec_capital_cost + ec_install_cost, 'M Euros')
print('Weather delays during export cable install', ec_delay, 'days')

Export cable total length:  90.0 km
Export cable install time:  18.0 days
Export cable install cost:  19.8 M Euros
Export cable capital cost:  49.5 M Euros
Export cable total cost:  69.3 M Euros
Weather delays during export cable install 1.8 days


### Offshore substation

In [17]:
project._phases['OffshoreSubstationInstallation'].detailed_output
# proj_df.loc[proj_df['phase']=='OffshoreSubstationInstallation']

{'OffshoreSubstationInstallation': {'Feeder_0_operational_efficiency': 1.0,
  'Feeder_1_operational_efficiency': 1.0,
  'Heavy_Lift_Vessel_operational_efficiency': 0.887495521782403,
  'Feeder_0_cargo_mass_utilization': 0.6431308308338641,
  'Feeder_1_cargo_mass_utilization': 0.6431308308338641,
  'Feeder_0_deck_space_utilization': 0.002,
  'Feeder_1_deck_space_utilization': 0.002}}

In [18]:
# Installation and capital costs 
oss_time = np.round(project._phases['OffshoreSubstationInstallation'].total_phase_time * (1/24) , 1) # Convert hours to days
oss_install_cost = np.round(project._phases['OffshoreSubstationInstallation'].total_phase_cost * 1e-6 * usd_to_euro, 1)
oss_capital_cost = np.round(project._phases['OffshoreSubstationDesign'].total_phase_cost * 1e-6 * usd_to_euro, 1)
oss_delay = np.round(oss_time * (1-project._phases['OffshoreSubstationInstallation'].detailed_output['OffshoreSubstationInstallation']['Heavy_Lift_Vessel_operational_efficiency']), 1)

print('Offshore substation install time: ', oss_time, 'days')
print('Offshore substation install cost: ', oss_install_cost, 'M Euros')
print('Offshore substation capital cost: ', oss_capital_cost, 'M Euros')
print('Offshore substation total cost: ', oss_capital_cost + oss_install_cost, 'M Euros')
print('Weather delays during substation install', oss_delay, 'days')

Offshore substation install time:  3.7 days
Offshore substation install cost:  1.7 M Euros
Offshore substation capital cost:  118.1 M Euros
Offshore substation total cost:  119.8 M Euros
Weather delays during substation install 0.4 days


In [19]:
proj_df.loc[(proj_df['phase']=='OffshoreSubstationInstallation')]

,action,agent,cable_lay_speed,cost,cost_multiplier,duration,hub_height,level,location,max_waveheight,max_windspeed,phase,phase_name,site_depth,time,transit_speed
6,Mobilize,Heavy Lift Vessel,NaN,1.085000e+06,1.0,120.000000,NaN,ACTION,NaN,NaN,NaN,OffshoreSubstationInstallation,NaN,NaN,0.000000,NaN
7,Mobilize,Feeder 0,NaN,0.000000e+00,0.5,72.000000,NaN,ACTION,NaN,NaN,NaN,OffshoreSubstationInstallation,NaN,NaN,0.000000,NaN
8,Mobilize,Feeder 1,NaN,0.000000e+00,0.5,72.000000,NaN,ACTION,NaN,NaN,NaN,OffshoreSubstationInstallation,NaN,NaN,0.000000,NaN
17,Transit,Heavy Lift Vessel,NaN,7.474444e+04,NaN,8.266667,NaN,ACTION,NaN,NaN,NaN,OffshoreSubstationInstallation,NaN,NaN,8.266667,NaN
20,Fasten Monopile,Feeder 0,NaN,0.000000e+00,NaN,12.000000,NaN,ACTION,NaN,NaN,NaN,OffshoreSubstationInstallation,OffshoreSubstationInstallation,NaN,12.000000,NaN
24,Fasten Topside,Feeder 0,NaN,0.000000e+00,NaN,2.000000,NaN,ACTION,NaN,NaN,NaN,OffshoreSubstationInstallation,OffshoreSubstationInstallation,NaN,14.000000,NaN
25,Queue,Feeder 1,NaN,0.000000e+00,NaN,14.000000,NaN,ACTION,NaN,NaN,NaN,OffshoreSubstationInstallation,NaN,NaN,14.000000,NaN
37,Fasten Monopile,Feeder 1,NaN,0.000000e+00,NaN,12.000000,NaN,ACTION,NaN,NaN,NaN,OffshoreSubstationInstallation,OffshoreSubstationInstallation,NaN,26.000000,NaN
40,Fasten Topside,Feeder 1,NaN,0.000000e+00,NaN,2.000000,NaN,ACTION,NaN,NaN,NaN,OffshoreSubstationInstallation,OffshoreSubstationInstallation,NaN,28.000000,NaN
44,Transit,Feeder 0,NaN,0.000000e+00,NaN,17.714286,NaN,ACTION,NaN,NaN,NaN,OffshoreSubstationInstallation,NaN,NaN,31.714286,NaN


### Onshore construction

In [20]:
proj_df.loc[(proj_df['phase']=='ExportCableInstallation') & (proj_df['action'] == 'Onshore Construction')]

,action,agent,cable_lay_speed,cost,cost_multiplier,duration,hub_height,level,location,max_waveheight,max_windspeed,phase,phase_name,site_depth,time,transit_speed
5,Onshore Construction,Onshore Construction,NaN,1.841086e+07,NaN,0.0,NaN,ACTION,Landfall,NaN,NaN,ExportCableInstallation,NaN,NaN,0.0,NaN


In [21]:
print('Onshore construction cost:', 1.84e7 * 1e-6 * usd_to_euro)

Onshore construction cost: 16.928


### Scour protection

In [28]:
project._phases['ScourProtectionInstallation'].detailed_output

{'ScourProtectionInstallation': {'SPI_Vessel_operational_efficiency': 0.9453309250716151}}

In [30]:
# Installation and capital costs 
scour_time = np.round(project._phases['ScourProtectionInstallation'].total_phase_time * (1/24) , 1) # Convert hours to days
scour_install_cost = np.round(project._phases['ScourProtectionInstallation'].total_phase_cost * 1e-6 * usd_to_euro, 1)
scour_capital_cost = np.round(project._phases['ScourProtectionDesign'].total_phase_cost * 1e-6 * usd_to_euro, 1)
scour_delay = np.round(scour_time * (1-project._phases['ScourProtectionInstallation'].detailed_output['ScourProtectionInstallation']['SPI_Vessel_operational_efficiency']), 1)


print('Scour protection install time: ', scour_time, 'days')
print('Scour protection install cost: ', scour_install_cost, 'M Euros')
print('Scour protection capital cost: ', scour_capital_cost, 'M Euros')
print('Scour protection total cost: ', scour_capital_cost + scour_install_cost, 'M Euros')
print('scour delay', scour_delay)

Scour protection install time:  72.5 days
Scour protection install cost:  8.2 M Euros
Scour protection capital cost:  4.5 M Euros
Scour protection total cost:  12.7 M Euros
scour delay 4.0


In [24]:
proj_df.loc[proj_df['phase']=='ScourProtectionInstallation']

,action,agent,cable_lay_speed,cost,cost_multiplier,duration,hub_height,level,location,max_waveheight,max_windspeed,phase,phase_name,site_depth,time,transit_speed
1,Mobilize,SPI Vessel,NaN,180000.000000,0.5,72.000000,NaN,ACTION,NaN,NaN,NaN,ScourProtectionInstallation,NaN,NaN,0.000000,NaN
9,Load SP Material,SPI Vessel,NaN,20000.000000,NaN,4.000000,NaN,ACTION,NaN,NaN,NaN,ScourProtectionInstallation,ScourProtectionInstallation,NaN,4.000000,NaN
36,Transit,SPI Vessel,NaN,103333.333333,NaN,20.666667,NaN,ACTION,NaN,NaN,NaN,ScourProtectionInstallation,NaN,NaN,24.666667,NaN
49,Drop SP Material,SPI Vessel,NaN,50000.000000,NaN,10.000000,NaN,ACTION,NaN,NaN,NaN,ScourProtectionInstallation,ScourProtectionInstallation,NaN,34.666667,NaN
50,Transit,SPI Vessel,NaN,1312.500000,NaN,0.262500,NaN,ACTION,NaN,NaN,NaN,ScourProtectionInstallation,NaN,NaN,34.929167,NaN
68,Drop SP Material,SPI Vessel,NaN,50000.000000,NaN,10.000000,NaN,ACTION,NaN,NaN,NaN,ScourProtectionInstallation,ScourProtectionInstallation,NaN,44.929167,NaN
98,Transit,SPI Vessel,NaN,103333.333333,NaN,20.666667,NaN,ACTION,NaN,NaN,NaN,ScourProtectionInstallation,NaN,NaN,65.595833,NaN
105,Load SP Material,SPI Vessel,NaN,20000.000000,NaN,4.000000,NaN,ACTION,NaN,NaN,NaN,ScourProtectionInstallation,ScourProtectionInstallation,NaN,69.595833,NaN
127,Transit,SPI Vessel,NaN,103333.333333,NaN,20.666667,NaN,ACTION,NaN,NaN,NaN,ScourProtectionInstallation,NaN,NaN,90.262500,NaN
136,Drop SP Material,SPI Vessel,NaN,50000.000000,NaN,10.000000,NaN,ACTION,NaN,NaN,NaN,ScourProtectionInstallation,ScourProtectionInstallation,NaN,100.262500,NaN
